In [69]:
import feedparser
import pandas as pd
import re
from collections import Counter, defaultdict
import html
from sentence_transformers import SentenceTransformer
import numpy as np
import sqlite3

/home/aizen/Code/PINE/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [70]:
feed_url = "https://samakal.com/rss"
feed = feedparser.parse(feed_url)

In [71]:
# Number of articles in the RSS feed
print(len(feed.entries))

100


In [72]:
for entry in feed.entries[:1]:
    print(sorted(entry.keys()))
    print("-" * 40)

['author', 'author_detail', 'authors', 'content', 'guidislink', 'id', 'link', 'links', 'published', 'published_parsed', 'summary', 'summary_detail', 'title', 'title_detail']
----------------------------------------


In [73]:
for entry in feed.entries[:2]:
    for key in entry.keys():
        print(f"{key}: {entry[key]}")
    print("=" * 100)

title: বিয়ের কথা বলে ডেকে নিয়ে সাবেক স্ত্রীকে গলা কেটে হত্যা
title_detail: {'type': 'text/plain', 'language': None, 'base': 'https://samakal.com/rss', 'value': 'বিয়ের কথা বলে ডেকে নিয়ে সাবেক স্ত্রীকে গলা কেটে হত্যা'}
links: [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://samakal.com/whole-country/article/331098/%E0%A6%AC%E0%A6%BF%E0%A7%9F%E0%A7%87%E0%A6%B0-%E0%A6%95%E0%A6%A5%E0%A6%BE-%E0%A6%AC%E0%A6%B2%E0%A7%87-%E0%A6%A1%E0%A7%87%E0%A6%95%E0%A7%87-%E0%A6%A8%E0%A6%BF%E0%A7%9F%E0%A7%87-%E0%A6%B8%E0%A6%BE%E0%A6%AC%E0%A7%87%E0%A6%95-%E0%A6%B8%E0%A7%8D%E0%A6%A4%E0%A7%8D%E0%A6%B0%E0%A7%80%E0%A6%95%E0%A7%87-%E0%A6%97%E0%A6%B2%E0%A6%BE-%E0%A6%95%E0%A7%87%E0%A6%9F%E0%A7%87-%E0%A6%B9%E0%A6%A4%E0%A7%8D%E0%A6%AF%E0%A6%BE'}]
link: https://samakal.com/whole-country/article/331098/%E0%A6%AC%E0%A6%BF%E0%A7%9F%E0%A7%87%E0%A6%B0-%E0%A6%95%E0%A6%A5%E0%A6%BE-%E0%A6%AC%E0%A6%B2%E0%A7%87-%E0%A6%A1%E0%A7%87%E0%A6%95%E0%A7%87-%E0%A6%A8%E0%A6%BF%E0%A7%9F%E0%A7%87-%E0%A6%B8%E0%A6%BE%E0%A6%AC%E0%A7%8

In [74]:
for entry in feed.entries[:1]:
    print(entry.title)
    print(entry.link)
    print(entry.author)
    print(entry.published_parsed)
    print(entry.summary)
    print(entry.content)
    print("=" * 100)

বিয়ের কথা বলে ডেকে নিয়ে সাবেক স্ত্রীকে গলা কেটে হত্যা
https://samakal.com/whole-country/article/331098/%E0%A6%AC%E0%A6%BF%E0%A7%9F%E0%A7%87%E0%A6%B0-%E0%A6%95%E0%A6%A5%E0%A6%BE-%E0%A6%AC%E0%A6%B2%E0%A7%87-%E0%A6%A1%E0%A7%87%E0%A6%95%E0%A7%87-%E0%A6%A8%E0%A6%BF%E0%A7%9F%E0%A7%87-%E0%A6%B8%E0%A6%BE%E0%A6%AC%E0%A7%87%E0%A6%95-%E0%A6%B8%E0%A7%8D%E0%A6%A4%E0%A7%8D%E0%A6%B0%E0%A7%80%E0%A6%95%E0%A7%87-%E0%A6%97%E0%A6%B2%E0%A6%BE-%E0%A6%95%E0%A7%87%E0%A6%9F%E0%A7%87-%E0%A6%B9%E0%A6%A4%E0%A7%8D%E0%A6%AF%E0%A6%BE
লালপুর (নাটোর) প্রতিনিধি
time.struct_time(tm_year=2025, tm_mon=12, tm_mday=25, tm_hour=15, tm_min=17, tm_sec=20, tm_wday=3, tm_yday=359, tm_isdst=0)
নাটোরের লালপুরে বিয়ের কথা বলে ডেকে নিয়ে তাম্মি আক্তার নামে এক নারীকে গলা কেটে হত্যা করেছেন তাঁর সাবেক স্বামী রবিন হোসেন। বৃহস্পতিবার বিকেলে উপজেলার শোভদিদারপাড়া এলাকার ইউএনও পার্ক-সংলগ্ন রেললাইনে এই নৃশংস ঘটনা ঘটে। এর পরপরই স্থানীয়রা অভিযুক্ত রবিনকে আটক করে পুলিশে সোপর্দ করেছে।

নিহত তাম্মি আক্তার উপজেলার নাবীরপাড়া গ্রামের জিল্লুর রহমানের ম

# Observed special strings
- &nbsp
- &rsquo
- &lsquo

In [75]:
data = []
for entry in feed.entries:
    data.append({
        'title': entry.get('title', ''),
        'link': entry.get('link', ''),
        'author': entry.get('author', ''),
        'published': entry.get('published', ''),
        'summary': entry.get('summary', '')
    })
df = pd.DataFrame(data)
df.head()

,title,link,author,published,summary
0,বিয়ের কথা বলে ডেকে নিয়ে সাবেক স্ত্রীকে গলা কেট...,https://samakal.com/whole-country/article/3310...,লালপুর (নাটোর) প্রতিনিধি,"Thu, 25 Dec 2025 21:17:20 +0600",নাটোরের লালপুরে বিয়ের কথা বলে ডেকে নিয়ে তাম্মি...
1,"দুই দিন পর মিলল শামছুর মরদেহ, ৩০ জনের নামে মামলা",https://samakal.com/whole-country/article/3310...,নোয়াখালী ও হাতিয়া প্রতিনিধি,"Thu, 25 Dec 2025 21:13:26 +0600",নোয়াখালীর হাতিয়ায় চর দখল নিয়ে সংঘর্ষ ও গোলাগুল...
2,পাংশায় গণপিটুনিতে ‘সম্রাট বাহিনী’র প্রধান নিহত,https://samakal.com/whole-country/article/3310...,রাজবাড়ী প্রতিনিধি,"Thu, 25 Dec 2025 21:08:34 +0600",চাঁদাবাজি করতে গিয়ে গণপিটুনিতে নিহত হয়েছেন &ls...
3,তারেক রহমানের স্বদেশ প্রত্যাবর্তনে যোগ দিতে যা...,https://samakal.com/whole-country/article/3310...,ভাঙ্গা (ফরিদপুর) প্রতিনিধি,"Thu, 25 Dec 2025 21:01:25 +0600",দীর্ঘ ১৭ বছর পর আজ বৃহস্পতিবার দেশে ফিরেন বিএন...
4,ব্রেকআপের পর নতুন প্রেমে বিল গেটসকন্যা,https://samakal.com/entertainment/article/3310...,বিনোদন ডেস্ক,"Thu, 25 Dec 2025 20:52:09 +0600",বিশ্বের অন্যতম ধনকুবের ও মাইক্রোসফটের সহপ্রতিষ...


In [76]:
print(df.info())
print(df.isnull().sum())
print(df.head(3))

for i, row in df.iterrows():
    print(f"Title: {row['title']}")
    print(f"Summary: {row['summary']}")
    print("="*80)
    if i >= 2:  # Show only first 3 for brevity
        break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      100 non-null    object
 1   link       100 non-null    object
 2   author     100 non-null    object
 3   published  100 non-null    object
 4   summary    100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB
None
title        0
link         0
author       0
published    0
summary      0
dtype: int64
                                               title  \
0  বিয়ের কথা বলে ডেকে নিয়ে সাবেক স্ত্রীকে গলা কেট...   
1   দুই দিন পর মিলল শামছুর মরদেহ, ৩০ জনের নামে মামলা   
2     পাংশায় গণপিটুনিতে ‘সম্রাট বাহিনী’র প্রধান নিহত   

                                                link  \
0  https://samakal.com/whole-country/article/3310...   
1  https://samakal.com/whole-country/article/3310...   
2  https://samakal.com/whole-country/article/3310...   

                        author           

In [77]:
# find all &*; patterns in summaries and count them
entity_counter = Counter()
entity_examples = defaultdict(list)

pattern = re.compile(r'&([a-zA-Z0-9]+);')

for summary in df['summary']:
    for match in pattern.finditer(summary):
        entity = match.group(0)
        entity_counter[entity] += 1
        # Store up to 3 example contexts for each entity
        if len(entity_examples[entity]) < 3:
            start = max(0, match.start()-20)
            end = min(len(summary), match.end()+20)
            context = summary[start:end]
            entity_examples[entity].append(context)

entities = set()
pattern = re.compile(r'&([a-zA-Z0-9]+);')
for summary in df['summary']:
    entities.update(pattern.findall(summary))
print(sorted(f'&{e};' for e in entities))

for entity, count in entity_counter.most_common():
    print(f'{entity} - {count}')
    for ex in entity_examples[entity]:
        print(f'  ex: ...{ex}...')
    print()

['&ldquo;', '&lrm;', '&lsquo;', '&mdash;', '&nbsp;', '&ndash;', '&radic;', '&rdquo;', '&rsquo;', '&zwj;', '&zwnj;']
&nbsp; - 428
  ex: ...ান উদ্ধার করা হয়েছে।&nbsp;

বর্তমানে তিনি পুলি...
  ex: ... হলে শেষকৃত্য করবেন।&nbsp;

পাংশা মডেল থানার ও...
  ex: ...দাবাজির মামলা রয়েছে।&nbsp;

রাজবাড়ীর পুলিশ সুপ...

&rsquo; - 258
  ex: ...ফে &lsquo;কোপা শামছু&rsquo;র (৫৫) মরদেহ উদ্ধার ...
  ex: ...ঠী &lsquo;কোপা শামছু&rsquo; বাহিনীর প্রধান হিসে...
  ex: ...quo;আলাউদ্দিন বাহিনী&rsquo; ও &lsquo;কোপা শামছু...

&lsquo; - 256
  ex: ...মো. শামছুদ্দিন ওরফে &lsquo;কোপা শামছু&rsquo;র (...
  ex: ...নি সন্ত্রাসী গোষ্ঠী &lsquo;কোপা শামছু&rsquo; বা...
  ex: ...লে নিতে গত মঙ্গলবার &lsquo;আলাউদ্দিন বাহিনী&rsq...

&ndash; - 30
  ex: ...িমিং প্ল্যাটফর্মগুলো&ndash;যেগুলো বিভিন্ন নানা ...
  ex: ... গীতিকবি, সংগীতায়োজক&ndash; সবাই এই মাধ্যমটিকে ...
  ex: ...ার সংস্থা স্যাকনিল্ক&ndash;এর তথ্য অনুযায়ী, ২০২...

&zwnj; - 9
  ex: ...ে জানা যায়, তার হৃদ্&zwnj;যন্ত্রের ক্রিয়া বন্ধ...
  ex: ...&rsquo;অরের পাওয়ার র&zwnj;্যাঙ

In [78]:
# clean RSS text: decode HTML entities (e.g., &nbsp;), drop HTML tags, normalize whitespace,

import html

def clean_text(value: str) -> str:
    if not isinstance(value, str):
        return ""
    text = html.unescape(value)
    # Remove zero-width / bidi markers that can cause invisible leading/trailing issues
    for z in ['\u200b', '\u200c', '\u200d', '\u200e', '\u200f']:
        text = text.replace(z, '')
    text = re.sub(r"<[^>]+>", " ", text)
    text = text.replace("\xa0", " ")
    text = re.sub(r"&[a-zA-Z0-9#]+;", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df_clean = df.copy()
text_columns = ['title', 'summary', 'author']
for col in text_columns:
    df_clean[col] = df_clean[col].fillna('').astype(str).map(clean_text)

rows_before = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['title', 'link']).reset_index(drop=True)

# Replace df with the cleaned version for downstream cells
df = df_clean
print(df.head(3)[['title', 'summary']])
print(f"Rows before={rows_before} after={len(df)}")


                                               title  \
0  বিয়ের কথা বলে ডেকে নিয়ে সাবেক স্ত্রীকে গলা কেট...   
1   দুই দিন পর মিলল শামছুর মরদেহ, ৩০ জনের নামে মামলা   
2     পাংশায় গণপিটুনিতে ‘সম্রাট বাহিনী’র প্রধান নিহত   

                                             summary  
0  নাটোরের লালপুরে বিয়ের কথা বলে ডেকে নিয়ে তাম্মি...  
1  নোয়াখালীর হাতিয়ায় চর দখল নিয়ে সংঘর্ষ ও গোলাগুল...  
2  চাঁদাবাজি করতে গিয়ে গণপিটুনিতে নিহত হয়েছেন ‘সম...  
Rows before=100 after=100


In [79]:
df

,title,link,author,published,summary
0,বিয়ের কথা বলে ডেকে নিয়ে সাবেক স্ত্রীকে গলা কেট...,https://samakal.com/whole-country/article/3310...,লালপুর (নাটোর) প্রতিনিধি,"Thu, 25 Dec 2025 21:17:20 +0600",নাটোরের লালপুরে বিয়ের কথা বলে ডেকে নিয়ে তাম্মি...
1,"দুই দিন পর মিলল শামছুর মরদেহ, ৩০ জনের নামে মামলা",https://samakal.com/whole-country/article/3310...,নোয়াখালী ও হাতিয়া প্রতিনিধি,"Thu, 25 Dec 2025 21:13:26 +0600",নোয়াখালীর হাতিয়ায় চর দখল নিয়ে সংঘর্ষ ও গোলাগুল...
2,পাংশায় গণপিটুনিতে ‘সম্রাট বাহিনী’র প্রধান নিহত,https://samakal.com/whole-country/article/3310...,রাজবাড়ী প্রতিনিধি,"Thu, 25 Dec 2025 21:08:34 +0600",চাঁদাবাজি করতে গিয়ে গণপিটুনিতে নিহত হয়েছেন ‘সম...
3,তারেক রহমানের স্বদেশ প্রত্যাবর্তনে যোগ দিতে যা...,https://samakal.com/whole-country/article/3310...,ভাঙ্গা (ফরিদপুর) প্রতিনিধি,"Thu, 25 Dec 2025 21:01:25 +0600",দীর্ঘ ১৭ বছর পর আজ বৃহস্পতিবার দেশে ফিরেন বিএন...
4,ব্রেকআপের পর নতুন প্রেমে বিল গেটসকন্যা,https://samakal.com/entertainment/article/3310...,বিনোদন ডেস্ক,"Thu, 25 Dec 2025 20:52:09 +0600",বিশ্বের অন্যতম ধনকুবের ও মাইক্রোসফটের সহপ্রতিষ...
...,...,...,...,...,...
95,বেপরোয়া জ্বালানি-ভোজ্যতেল ছিনতাই ও চোর চক্র,https://samakal.com/whole-country/article/3310...,"আমিনুল ইসলাম খান রানা, সিরাজগঞ্জ","Thu, 25 Dec 2025 07:56:56 +0600",ঢাকা-রংপুর-পাবনা মহাসড়ককে বলা হয় দেশের উত্তরাঞ...
96,দিপু দাস হত্যার বিচার দ্রুত না হলে কঠোর আন্দোলন,https://samakal.com/whole-country/article/3310...,"নিজস্ব প্রতিবেদক, ময়মনসিংহ","Thu, 25 Dec 2025 07:55:16 +0600",ময়মনসিংহের ভালুকা উপজেলায় পোশাক কারখানার শ্রমি...
97,"আটক নেই, হয়নি মামলা ও অস্ত্র উদ্ধার",https://samakal.com/whole-country/article/3310...,নোয়াখালী ও হাতিয়া প্রতিনিধি,"Thu, 25 Dec 2025 07:53:55 +0600",নোয়াখালীর দ্বীপ উপজেলা হাতিয়ায় চর দখল নিয়ে দুপ...
98,ভোটের আগে রাউজানে হিন্দুদের মধ্যে আতঙ্ক,https://samakal.com/whole-country/article/3309...,"সারোয়ার সুমন, চট্টগ্রাম","Thu, 25 Dec 2025 07:52:42 +0600",নির্বাচন ঘনিয়ে আসতেই চট্টগ্রামের রাউজানে হাজির...


In [80]:
print(df.iloc[0])

title        বিয়ের কথা বলে ডেকে নিয়ে সাবেক স্ত্রীকে গলা কেট...
link         https://samakal.com/whole-country/article/3310...
author                                লালপুর (নাটোর) প্রতিনিধি
published                      Thu, 25 Dec 2025 21:17:20 +0600
summary      নাটোরের লালপুরে বিয়ের কথা বলে ডেকে নিয়ে তাম্মি...
Name: 0, dtype: object


## Store vectors in SQLite
Steps:
1. Load the multilingual encoder `sentence-transformers/paraphrase-multilingual-mpnet-base-v2` (handles Bangla).
2. Build a clean text field from title + summary, encode to normalized vectors.
3. Create a SQLite table with metadata + `language='bangla'` and persist embeddings as float32 blobs.
4. Commit once to keep inserts atomic; rerun to refresh the table.


In [81]:
model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(model_name)

def build_text(row):
    parts = [row.get('title', ''), row.get('summary', '')]
    return ' '.join(p for p in parts if p).strip()

texts = df.apply(build_text, axis=1).tolist()
embeddings = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

conn = sqlite3.connect('samakal_vectors.sqlite')
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    summary TEXT,
    link TEXT,
    published TEXT,
    language TEXT,
    embedding BLOB
)
''')
cur.execute('DELETE FROM articles')  # refresh on reruns

def to_blob(vec: np.ndarray) -> bytes:
    return vec.astype('float32').tobytes()

rows = []
for (_, row), emb in zip(df.iterrows(), embeddings):
    published = None
    if pd.notna(row.get('published')):
        published = pd.to_datetime(row['published']).isoformat()
    rows.append((row.get('title', ''), row.get('summary', ''), row.get('link', ''), published, 'bangla', to_blob(emb)))

cur.executemany(
    'INSERT INTO articles (title, summary, link, published, language, embedding) VALUES (?, ?, ?, ?, ?, ?)',
    rows
)
conn.commit()
conn.close()

print(f'Inserted {len(rows)} rows into samakal_vectors.sqlite with language=bangla')


Inserted 100 rows into samakal_vectors.sqlite with language=bangla
